In [50]:
import pandas as pd
import numpy as np
import pickle
import glob
import os

In [51]:
# 모든 리뷰 데이터를 이 dir에 넣어두기
DATA_ROOT = './QnA_data/'

# category id table이 저장되어있는 dir
LABEL_TABLE_ROOT = './LABEL_TABLE/'

# 파싱한 결과를 json 파일로 저장할 dir
SAVE_ROOT = './parsed_QnA/'

# 만약 SAVE_ROOT dir이 존재하지 않으면 dir 생성
if not os.path.exists(SAVE_ROOT): os.mkdir(SAVE_ROOT)

# file list 불러오기
file_list = glob.glob(DATA_ROOT+'*.*')

# id table 불러오기
label_table = pd.read_excel(LABEL_TABLE_ROOT+'label_table.xlsx')

In [52]:
class parse_QnA:
    def __init__(self, file_list):
        self.file_list = file_list
        self.category_ids = []
        self.data = []
        for file in self.file_list:
            df = pd.read_excel(file)
            self.data.append(df)
            try : self.category_ids.append(list(np.unique(df.id.dropna().astype(np.int64))))
            except : self.category_ids.append(list(np.unique(df.Id.dropna().astype(np.int64))))
    
    def get_category_ids(self):
        result = []

        for ids in self.category_ids:
            result.extend(ids)
        
        return np.unique(result)
    
    def check_unincluded_category_ids(self, label_table):
        cur_labels = set(self.get_category_ids())
        whole_labels = set(label_table.label)
        unincluded_labels = whole_labels.difference(cur_labels)

        if len(unincluded_labels) == 0: print("All labels are included!")
        else : print(f"unincluded labels : {unincluded_labels}")

    def parse(self):
        self.df_Q = pd.DataFrame()
        self.Dict_A = {}

        questions = []
        id_per_question = []

        for df, id_list in zip(self.data, self.category_ids):
            for id in id_list:
                try : arr = list(np.unique(df.Q[df.id == id]))
                except : arr = list(np.unique(df.Q[df.Id == id]))
                questions.extend(arr)
                for _ in range(len(arr)):
                    id_per_question.append(id)

                try : answers = list(np.unique(df.A[df.id == id].dropna()))
                except : answers = list(np.unique(df.A[df.Id == id].dropna()))
                self.Dict_A[str(id)] = answers
        
        if len(questions) == len(id_per_question):
            self.df_Q['Question'] = questions
            self.df_Q['id'] = id_per_question
            print("Data Parsing Complete!")
        else:
            print("[Number of Qustions] And [Number of id] must be same!")
    
    def get_parse(self):
        try : return self.df_Q, self.Dict_A
        except : print("Should Use 'parse()' method first")
    
    def to_pickle(self, root='./'):
        self.df_Q.to_pickle(root+'question_data.pkl')
        
        with open(root+'answer_data.pkl', 'wb') as f:
            pickle.dump(self.Dict_A, f, pickle.HIGHEST_PROTOCOL)
        
        print("pickle file saved!")
    
    def question_to_csv(self, root='./'):
        self.df_Q.to_csv(root+'question_data.csv', index=False)

        print("csv file saved!")

In [53]:
# parse data 넣기
parser = parse_QnA(file_list)

# Data에 포함된 category_ids
category_ids = parser.get_category_ids()

# data에 모든 id가 포함되어있는지 확인
parser.check_unincluded_category_ids(label_table=label_table)

# data parse
parser.parse()

# parsing한 data 
parser.to_pickle(root=SAVE_ROOT)

# question data to csv
parser.question_to_csv(root=SAVE_ROOT)

All labels are included!
Data Parsing Complete!
pickle file saved!
csv file saved!


In [54]:
category_ids

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153], dtype=int64)

In [55]:
question = pd.read_pickle(SAVE_ROOT+'question_data.pkl')

question[question.id == 30]

,Question,id
1005,취업과진로 강의평가가 궁금함,30
1006,취업과진로 강의평이 궁금해,30
1007,취업과진로 과제 많아?,30
1008,취업과진로 과제 많음?,30
1009,취업과진로 꿀교양이야?,30
1010,취업과진로 수업 어때?,30
1011,취업과진로 어때?,30
1012,취업과진로 황용식 교수님 강의평가가 궁금함,30
1013,취업과진로 황용식 교수님 강의평이 궁금해,30
1014,취업과진로 황용식 교수님 과제 많아?,30


In [56]:
question_csv = pd.read_csv(SAVE_ROOT+'question_data.csv')

question[question.id == 30]

,Question,id
1005,취업과진로 강의평가가 궁금함,30
1006,취업과진로 강의평이 궁금해,30
1007,취업과진로 과제 많아?,30
1008,취업과진로 과제 많음?,30
1009,취업과진로 꿀교양이야?,30
1010,취업과진로 수업 어때?,30
1011,취업과진로 어때?,30
1012,취업과진로 황용식 교수님 강의평가가 궁금함,30
1013,취업과진로 황용식 교수님 강의평이 궁금해,30
1014,취업과진로 황용식 교수님 과제 많아?,30


In [57]:
with open(SAVE_ROOT+'answer_data.pkl', 'rb') as f:
    answer = pickle.load(f)

answer['30']

['2학점 패논패 과목으로 중간/기말에는 보고서를 작성하는데 각각 30분 걸리는데 출석 제대로 하고 보고서 2번 작성하면 깔끔하게 패스 받을 수 있습니다.',
 '개꿀 2학점짜리 패논패. 무조건 듣는 거 추천. 시험은 없고 중간 기말 대채 과제는 있음',
 '과제 제출만 잘하면 패스합니다~ 과제는 중간 2번, 기말 2번 제출하는데 부담없이 할 수 있어요',
 '꿀강의 그자체 안들으면 ㅂ ㅏ 보 임!  그리고 열심히 들으면 꽤 유용한 내용을 많이 알아감',
 '다른 강의평 보면 알겠지만, 개꿀 과목. 과제만 제때 내면 P받고 학점도 받고',
 '들으시는 분들 취창업이랑 같이들으면 좋아요 추천합니다~시험대체 과제 겹치는 거 많아요',
 '말해 뭐해 ㅎㅎ 무조건 들어야하는 꿀강~! 상담하는게 과제임',
 '시험 안보는 패논패 과목이라 정말 좋아요. 가능하다면 꼭 듣는걸 추천드려요!',
 '아주 졸아요 짱짱맨 무야호~~~아니 ㄹㅇ 개꿀임 하루만에도 ㅆㄱㄴ',
 '역시 취진 최고!! 까먹지 말고 할것만 다하면됨!!',
 '완전 꿀 강의에다 패논패라서 너무 좋았습니당 !!!!',
 '일단 신청하세요. 후회 없을겁니다!! 중간 기말도 그렇게 막 시간을 많이 요구하는 과제도 아니여서 굉장히 좋았습니다',
 '중간, 기말 대체 과제고 어렵지 않아서 꿀강의임',
 '취창업과 함께 들었는데 학점받기도 좋고 부담스럽지 않았음',
 '패논패 2학점 교선1이라 매우 꿀교양으로 생각한다. 중간이랑 기말은 과제대체형식인데 다른 교양들에 비해서도 매우 쉬운편이라 교양비면 듣는것추천함.',
 '패논패고 꿀강의 플러스 취업관련 정보도 얻어가서 유익함']